In [1]:
#To Check Sapark is working
sc

In [5]:
sc.version

'1.6.0'

In [2]:
#import OS for navigation file and folder.In Jupyter ls, dir, cd, pwd work well no need to import os 
import os

In [3]:
os.getcwd()

'/home/genius'

In [4]:
pwd

'/home/genius'

In [6]:
#Loading Uber Data
uber = sc.textFile("uber_reduce.csv")

In [7]:
#See First Row
uber.first()

'dispatching_base_number,date,active_vehicles,trips'

In [8]:
#See some more rows
uber.take(5)

['dispatching_base_number,date,active_vehicles,trips',
 'B02617,2/5/2015,1524,14499',
 'B02682,2/5/2015,1418,13782',
 'B02598,2/5/2015,1179,11609',
 'B02512,2/5/2015,264,2022']

In [11]:
#Count total number of rows
uber.count()

107

In [9]:
#Do some cleaning on Data like remove first row and spliting save all in new RDD name uber_final
uber_final = uber.filter(lambda row: "date" not in row).map(lambda line: line.split(","))

In [10]:
uber_final.take(5)

[['B02617', '2/5/2015', '1524', '14499'],
 ['B02682', '2/5/2015', '1418', '13782'],
 ['B02598', '2/5/2015', '1179', '11609'],
 ['B02512', '2/5/2015', '264', '2022'],
 ['B02765', '2/5/2015', '355', '3011']]

In [13]:
#After removing first row
uber_final.count()

106

In [22]:
#See type of uber and uber_final
type(uber)

pyspark.rdd.RDD

In [23]:
type(uber_final)

pyspark.rdd.PipelinedRDD

In [24]:
#Want to reuse uber_final RDD use cache()
uber_final.cache()

PythonRDD[9] at RDD at PythonRDD.scala:43

In [25]:
#To print first Column i.e dispatching_base_number of uber_final RDD
for col in uber_final.take(uber_final.count()): print(col[0])

B02617
B02682
B02598
B02512
B02765
B02764
B02617
B02765
B02598
B02512
B02764
B02682
B02598
B02512
B02617
B02682
B02764
B02765
B02764
B02765
B02598
B02617
B02682
B02512
B02617
B02682
B02598
B02764
B02512
B02765
B02764
B02512
B02617
B02765
B02682
B02598
B02617
B02764
B02512
B02598
B02765
B02682
B02617
B02512
B02682
B02765
B02598
B02764
B02617
B02682
B02764
B02765
B02512
B02598
B02764
B02512
B02598
B02765
B02617
B02682
B02682
B02764
B02617
B02765
B02512
B02598
B02598
B02512
B02617
B02764
B02682
B02765
B02764
B02512
B02682
B02617
B02598
B02765
B02598
B02682
B02617
B02765
B02764
B02512
B02598
B02512
B02682
B02764
B02765
B02617
B02764
B02617
B02598
B02682
B02765
B02512
B02598
B02682
B02765
B02617
B02512
B02764
B02512
B02617
B02682
B02764


In [26]:
#Display total number of distinct dispatching_base_number 
uber_final.map(lambda col: col[0]).distinct().count()

6

In [27]:
#Display name of distinct dispatching_base_number
uber_final.map(lambda col: col[0]).distinct().collect()

['B02598', 'B02682', 'B02765', 'B02617', 'B02764', 'B02512']

In [28]:
#Display total no of times dispatching_base_number - "B02598" occurs to do this we use filter()
uber_final.filter(lambda row: "B02598" in row).count()

17

In [29]:
#Filter and create new RDD "baseB02598"
baseB02598 = uber_final.filter(lambda row: "B02598" in row)

In [30]:
#Display first 5 rows
baseB02598.take(5)

[['B02598', '2/5/2015', '1179', '11609'],
 ['B02598', '2/6/2015', '1181', '11897'],
 ['B02598', '2/7/2015', '1031', '10512'],
 ['B02598', '2/8/2015', '923', '8129'],
 ['B02598', '2/9/2015', '976', '8135']]

In [38]:
#Count where trip of baseB02598 is greater than 10K
baseB02598.filter(lambda col: int(col[3]) > 10000 ).count()

11

In [39]:
#Display where trip of baseB02598 is greater than 10K
baseB02598.filter(lambda col: int(col[3]) > 10000).collect()

[['B02598', '2/5/2015', '1179', '11609'],
 ['B02598', '2/6/2015', '1181', '11897'],
 ['B02598', '2/7/2015', '1031', '10512'],
 ['B02598', '2/11/2015', '1115', '10034'],
 ['B02598', '2/12/2015', '1181', '11640'],
 ['B02598', '2/13/2015', '1216', '13062'],
 ['B02598', '2/14/2015', '1111', '12678'],
 ['B02598', '2/15/2015', '1003', '11517'],
 ['B02598', '2/19/2015', '1127', '11739'],
 ['B02598', '2/20/2015', '1186', '12758'],
 ['B02598', '2/21/2015', '1044', '12132']]

In [40]:
#Some trailer of Key,value pair 
#Display total number of trip of each Base
uber_final.map(lambda kp: (kp[0],int(kp[3]))).reduceByKey(lambda k,v: k + v).collect()

[('B02598', 182613),
 ('B02682', 238722),
 ('B02765', 81875),
 ('B02617', 247329),
 ('B02764', 656110),
 ('B02512', 33136)]

In [43]:
#Top 5 Busiest Base according to trip
uber_final.map(lambda kp: (kp[0],int(kp[3]))).reduceByKey(lambda k,v: k + v).takeOrdered(5,key = lambda x: -x[1])

[('B02764', 656110),
 ('B02617', 247329),
 ('B02682', 238722),
 ('B02598', 182613),
 ('B02765', 81875)]

In [44]:
#Some other concept like how to create own RDD by using parallelize()
own_Rdd1 = sc.parallelize(range(1,10))

In [45]:
type(own_Rdd1)

pyspark.rdd.PipelinedRDD

In [46]:
own_Rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [48]:
#map() vs flatmap(),faltmap() flaten the data set i.e remove hierarchy in data set which is in json data
#map() on own RDD
sc.parallelize([2,3,4]).map(lambda x:[x,x,x]).collect()

[[2, 2, 2], [3, 3, 3], [4, 4, 4]]

In [53]:
#flatMap():- It is helpful when dealing with data set which has some hierarchy to it. Ultimately remove hierarchy
sc.parallelize([2,3,4]).flatMap(lambda x: [x,x,x]).collect()

[2, 2, 2, 3, 3, 3, 4, 4, 4]

In [52]:
#To see default Parallelism by default two partitions on each Core
print(sc.defaultParallelism)

2


In [61]:
#Some sets operation on RDD like combine, compare and distinct 
own_Rdd2 = sc.parallelize(range(5,15))

In [62]:
own_Rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [63]:
own_Rdd2.collect()

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [66]:
#Union() operation, Union() take one argument  and repetition of elements allowed 
own_Rdd1.union(own_Rdd2).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [67]:
#Intersection(),It display common elements in RDDs,syntax:- takes one Rdd as argument
own_Rdd1.intersection(own_Rdd2).collect()

[8, 5, 9, 6, 7]

In [68]:
#Display Distinct elements in Rdds
own_Rdd1.union(own_Rdd2).distinct().collect()

[8, 12, 4, 1, 13, 5, 9, 2, 14, 10, 6, 11, 3, 7]